In [ ]:
import numpy as np
import geopandas as gpd


import xarray as xr
from matplotlib import pyplot as plt
%matplotlib inline

from xhistogram.xarray import histogram
import dask.dataframe as dd
#import cartopy.crs as ccrs
#import cartopy.feature as cfeature

import mitequinox.utils as ut
from mitequinox.plot import *
import mitequinox.parcels as pa
from xmitgcm import llcreader

from scipy import signal
import scipy.ndimage as im
import scipy.optimize as opt

from sympy import Symbol, pi, atan, factor, lambdify

import mitequinox.plot as pl
import mitequinox.sigp as sp


from fsspec.implementations.local import LocalFileSystem

In [ ]:
from dask.distributed import Client, LocalCluster
#
cluster = LocalCluster()
#
from dask_jobqueue import PBSCluster
#cluster = PBSCluster(walltime='02:00:00') #processes=7, cores=7
#w = cluster.scale(jobs=5)
#
client = Client(cluster)

In [ ]:
ds =  xr.open_zarr(ut.work_data_dir+'filtered_itide/SSU_bandpassed_filtered.zarr')
grd = ut.load_grd(consolidated=False)[['XC', 'YC', 'Depth']]#.persist()
grd = grd.isel(j=slice(0,None,4),i=slice(0,None,4))
ds['XC'] = grd.XC
ds['YC'] = grd.YC

In [ ]:
pl.plot_pretty(ds.SSU_demodulated_real,cmap='magma',vmax=0.01,land=False
              )

In [ ]:
ds['Depth'] = grd.Depth
ds['lon'] = 0*ds.face+grd.XC
ds['lat'] = 0*ds.face+grd.YC

In [ ]:
ds_GS = ds.where((lon<-60) & (lon>-90) & (lat<40) & (lat>20))
ds_atl = ds.where((lon<0) & (lon>-60) & (lat<-10) & (lat>-30))

In [1]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
def plot_zoom(ds,colmap='RdBu_r',vlim=(-0.01,0.01),axes=111,**kwargs):
    crs = ccrs.Robinson()
    #fig = plt.figure(figsize=figsize)
    ax = fig.add_subplot(axes,projection=crs)
    ds.plot(vmax=vlim[1],vmin=vlim[0],y='lat',ax=ax,
            transform=ccrs.PlateCarree(),cmap=colmap,label=False,**kwargs,
           cbar_kwargs={'fraction':0.02})
    ax.gridlines(draw_labels=True)
    ax.add_feature(cfeature.COASTLINE)
    ax.add_feature(cfeature.LAND)
    return fig,ax

In [ ]:
cluster.close()